## Global Terrorism Attribution Predictor

In this project I...

Imports and field variables

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
data_path = "./data"

Download Global Terrorism Database from Kaggle

In [2]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files("START-UMD/gtd", quiet=False, path=data_path)

gtd.zip: Skipping, found more recently modified local copy (use --force to force download)


Read data into Pandas DataFrame

In [3]:
# skipcols=lambda x: x not in ["extended", "addnotes", "scite1", "scite2", "scite3", "dbsource"]
usecols=["iyear",
         "suicide",
         "imonth",
         "country",
         "region",
         "provstate",
         "city",
         "alternative",
         "attacktype1",
         "targtype1",
         "natlty1",
         "gname",
         "weaptype1",
         "ransom"]

data = pd.read_csv(data_path + "/gtd.zip",
                        encoding = "ISO-8859-1",
                        compression='zip',
                        usecols=usecols)

set dTypes

In [4]:
# data = pd.DataFrame(data.dropna())
# data = data.fillna(value={0})
data['alternative'] = data.alternative.astype('category')
data['suicide'] = data.suicide.astype('category')
data['iyear'] = data.iyear.astype('category')
data['imonth'] = data.imonth.astype('category')
data['country'] = data.country.astype('category')
data['region'] = data.region.astype('category')
data['provstate'] = data.provstate.astype('str').astype('category')
data['city'] = data.city.astype('str').astype('category')
data['attacktype1'] = data.attacktype1.astype('category')
data['targtype1'] = data.targtype1.astype('category')
data['natlty1'] = data.natlty1.astype('category')
data['weaptype1'] = data.weaptype1.astype('category')
data['ransom'] = data.ransom.astype('category')

In [5]:
pd.set_option('display.max_columns', None) 
data.head()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
0,1970,7,58,2,nan,Santo Domingo,NaN,0,1,14,58.0,MANO-D,13,0.0
1,1970,0,130,1,Federal,Mexico city,NaN,0,6,7,21.0,23rd of September Communist League,13,1.0
2,1970,1,160,5,Tarlac,Unknown,NaN,0,1,10,217.0,Unknown,13,0.0
3,1970,1,78,8,Attica,Athens,NaN,0,3,7,217.0,Unknown,6,0.0
4,1970,1,101,4,Fukouka,Fukouka,NaN,0,7,7,217.0,Unknown,8,0.0


In [6]:
data.describe()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
count,181691,181691,181691,181691,181691,181691,29011.0,181691,181691,181691,180132.0,181691,181691,77381.0
unique,47,13,205,12,2856,36675,5.0,2,9,22,215.0,3537,12,3.0
top,2014,5,95,10,Baghdad,Unknown,1.0,0,3,14,95.0,Unknown,6,0.0
freq,16903,16875,24636,50474,7645,9775,23410.0,175058,88255,43511,24113.0,82782,92426,74684.0


Filter out unattributed attacks

In [7]:
attributed = data.loc[data['gname'] != "Unknown"]

In [8]:
# attributed = pd.DataFrame(attributed.dropna())
# attributed = attributed.iloc[:10000]
attributed = attributed.sample(frac=1)
attributed.head()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
59195,1995,5,92,6,Jammu and Kashmir,Srinagar,NaN,0,8,20,92.0,Muslim Separatists,9,0.0
38330,1989,5,45,3,Santander,Bucaramanga,NaN,0,1,2,45.0,National Liberation Army of Colombia (ELN),5,0.0
40900,1989,12,186,6,Southern,Hakmana,NaN,0,2,14,186.0,People's Liberation Front (JVP),5,0.0
73810,2002,1,45,3,Valle del Cauca,Cali,2.0,0,2,10,45.0,Gunmen,5,NaN
38291,1989,5,8,11,Cuanza Norte,Unknown,NaN,0,3,21,8.0,National Union for the Total Independence of A...,6,0.0


In [9]:
to_fit = attributed.select_dtypes(include=['object'])
le = preprocessing.LabelEncoder()
attributed = attributed.apply(le.fit_transform)
attributed.head()

,iyear,imonth,country,region,provstate,city,alternative,suicide,attacktype1,targtype1,natlty1,gname,weaptype1,ransom
59195,24,5,70,5,783,22168,36226,0,7,19,69,2134,7,1
38330,19,5,31,2,1582,3978,51037,0,0,1,31,2201,3,1
40900,19,12,143,5,1691,9140,51036,0,1,13,145,2458,3,1
73810,31,1,31,2,1851,4425,1,0,1,9,31,1329,3,26161
38291,19,5,4,10,453,23975,51035,0,2,20,3,2239,4,1


In [10]:
# 1. INSTANTIATE
# enc = preprocessing.OneHotEncoder()

# # 2. FIT
# enc.fit(attributed)

# 3. Transform
# onehotlabels = enc.transform(attributed).toarray()
# attributed = pd.get_dummies(attributed)
# attributed.shape

Making a copy of the attributeddataframe which does not contain the dependent variable

In [11]:
independent_vars = attributed.copy()
independent_vars = independent_vars[independent_vars.columns.drop(list(independent_vars.filter(regex='gname')))]
independent_vars.shape

(98909, 13)

Making a dataframe with of the dependent variables (organizations to which the attacks were attribued)

In [12]:
dependent_vars = pd.DataFrame(attributed.loc[:,list(attributed.filter(regex='gname'))])
dependent_vars.shape

(98909, 1)

Split the data into training and test sets

In [13]:
independant_size = independent_vars.shape
dependant_size = dependent_vars.shape
if independant_size[0] != dependant_size[0]:
    raise ValueError('Independent and Dependant DFs do not match')

split_size = int(independant_size[0] * 0.7)

training_independant_vars = independent_vars.iloc[:split_size]
test_independant_vars = independent_vars.iloc[split_size:]
print(training_independant_vars.shape)
print(test_independant_vars.shape)

training_dependant_vars = dependent_vars.iloc[:split_size]
test_dependant_vars = dependent_vars.iloc[split_size:]

print(training_dependant_vars.shape)
print(test_dependant_vars.shape)

(69236, 13)
(29673, 13)
(69236, 1)
(29673, 1)


TODO: Implement Training model

In [14]:
reg = RandomForestClassifier(verbose=10, n_estimators=35, warm_start=True)
reg.fit(training_independant_vars, training_dependant_vars.values.ravel())
# reg.fit(train_fold, train_y.values.ravel())
predictions = reg.predict(test_independant_vars)
total = accuracy_score(test_dependant_vars, predictions)
print("Accuracy is {}%".format(total * 100))


building tree 1 of 35


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


building tree 2 of 35


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.6s remaining:    0.0s


building tree 3 of 35


...

building tree 9 of 35


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   28.7s remaining:    0.0s


building tree 10 of 35
building tree 11 of 35
building tree 12 of 35
building tree 13 of 35
building tree 14 of 35
building tree 15 of 35
building tree 16 of 35
building tree 17 of 35
building tree 18 of 35
building tree 19 of 35
building tree 20 of 35
building tree 21 of 35
building tree 22 of 35
building tree 23 of 35
building tree 24 of 35
building tree 25 of 35
building tree 26 of 35
building tree 27 of 35
building tree 28 of 35
building tree 29 of 35
building tree 30 of 35
building tree 31 of 35
building tree 32 of 35
building tree 33 of 35
building tree 34 of 35
building tree 35 of 35


[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  1.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    6.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    7.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   25.9s finished


Accuracy is 76.50052236039497%
